In [1]:
import os
import cv2
import numpy as np
import json
from tqdm import tqdm

In [2]:
# Resizing과 Padding 시행할 이미지 데이터셋 경로
data_root = 'D:\\Falldown\\Dataset\\Sorted_Dataset'

train_root = f'{data_root}\\Train\\Image'
val_root = f'{data_root}\\Val\\Image'
test_root = f'{data_root}\\Test\\Image'

# YOLO 라벨 형식으로 변환할 JSON 데이터셋 경로
train_json = f'{data_root}\\Train\\Label'
val_json = f'{data_root}\\Val\\Label'
test_json = f'{data_root}\\Test\\Label'

In [6]:
resized_data_root = 'D:\\Falldown\\Dataset\\Resized_Dataset'

resized_train_root = f'{resized_data_root}\\Train\\images'
resized_val_root = f'{resized_data_root}\\Val\\images'
resized_test_root = f'{resized_data_root}\\Test\\images'

# Resizing 반영한 JSON 데이터셋 경로
resized_train_json = f'{resized_data_root}\\Train\\json_resized'
resized_val_json = f'{resized_data_root}\\Val\\json_resized'
resized_test_json = f'{resized_data_root}\\Test\\json_resized'

# Resizing과 정규화를 시행한 YOLO Label 데이터셋 경로
yolo_train_label = f'{resized_data_root}\\Train\\labels'
yolo_val_label = f'{resized_data_root}\\Val\\labels'
yolo_test_label = f'{resized_data_root}\\Test\\labels'

In [ ]:
# 이미지 리사이징 및 패딩
def resize_padding_images(input_folders, output_folders, target_size=(640, 640)):
    for input_root, output_root in zip(input_folders, output_folders):
        if not os.path.exists(output_root):
            os.makedirs(output_root)

        image_files = [file for file in os.listdir(input_root) if file.endswith('.jpg')]
        skipped_image = 0  # 건너뛴 이미지 개수 카운터
        for file in tqdm(image_files, desc=f"{input_root} 이미지 resize 및 padding 진행 중", unit="file"):
            file_path = os.path.join(input_root, file)
            output_path = os.path.join(output_root, file)
            if os.path.exists(output_path):
                skipped_image += 1  # 건너뛴 파일 카운트
                continue  # 이미 존재하는 경우, 다음 루프 항목으로 넘어감

            # 이미지 읽기
            image = cv2.imread(file_path)
            if image is None:
                try:
                    image = cv2.imdecode(np.fromfile(file_path, np.uint8), cv2.IMREAD_COLOR)
                except Exception as e:
                    print(f"Error reading image {file_path}: {e}")
                    continue

            # 이미지 리사이징 및 패딩
            try:
                h, w = image.shape[:2]
                scale = min(target_size[0] / w, target_size[1] / h)
                new_w, new_h = int(w * scale), int(h * scale)
                resized_image = cv2.resize(image, (new_w, new_h), interpolation=cv2.INTER_LINEAR)
                pad_w, pad_h = (target_size[0] - new_w) // 2, (target_size[1] - new_h) // 2
                padded_image = cv2.copyMakeBorder(resized_image, pad_h, target_size[1] - new_h - pad_h,
                                                  pad_w, target_size[0] - new_w - pad_w, cv2.BORDER_CONSTANT, value=[128, 128, 128])

                # 결과 저장
                if not cv2.imwrite(output_path, padded_image):
                    print(f"Error: 이미지 저장 실패 - {output_path}")
            except Exception as e:
                print(f"Warning: {file_path} 경로의 이미지 처리에 오류가 발생하였습니다. {e}")

        # 루프 종료 후 요약 메시지 출력
        print(f"'{input_root}'의 총 {len(image_files)}개 이미지 중 이미 존재하는 {skipped_image}개 이미지를 건너뛰었습니다.")

In [18]:
try:
    # 이미지 Resizing 및 Padding 시행
    print("Image resizing과 padding 시작")
    resize_padding_images(
        [train_root, val_root, test_root],
        [resized_train_root, resized_val_root, resized_test_root],
        target_size=(640, 640)
    )
except KeyboardInterrupt:
    print("KeyboardInterrupt: 사용자가 프로세스를 중단하였습니다.")

Image resizing과 padding 시작


D:\Falldown\Dataset\Sorted_Dataset\Train\Image 이미지 resize 및 padding 진행 중:   0%|          | 0/76130 [00:00<?, ?file/s]

D:\Falldown\Dataset\Sorted_Dataset\Train\Image 이미지 resize 및 padding 진행 중: 100%|██████████| 76130/76130 [00:06<00:00, 12418.68file/s]


'D:\Falldown\Dataset\Sorted_Dataset\Train\Image'의 총 76130개 이미지 중 76130개 이미지를 건너뛰었습니다.


D:\Falldown\Dataset\Sorted_Dataset\Val\Image 이미지 resize 및 padding 진행 중: 100%|██████████| 9440/9440 [00:00<00:00, 12785.41file/s]


'D:\Falldown\Dataset\Sorted_Dataset\Val\Image'의 총 9440개 이미지 중 9440개 이미지를 건너뛰었습니다.


D:\Falldown\Dataset\Sorted_Dataset\Test\Image 이미지 resize 및 padding 진행 중: 100%|██████████| 9580/9580 [00:00<00:00, 12771.85file/s]

'D:\Falldown\Dataset\Sorted_Dataset\Test\Image'의 총 9580개 이미지 중 9580개 이미지를 건너뛰었습니다.


In [ ]:
# JSON 라벨을 YOLO 라벨로 변환 및 Resized JSON 저장
def convert_json_to_yolo_and_resized(input_folder, resized_json_folder, yolo_label_folder, original_size=(3840, 2160), new_size=(640, 640)):
    if not os.path.exists(resized_json_folder):
        os.makedirs(resized_json_folder)
    if not os.path.exists(yolo_label_folder):
        os.makedirs(yolo_label_folder)

    json_files = [file for file in os.listdir(input_folder) if file.endswith('.json')]
    skipped_json_label = 0 
    processed_json_label = 0
    for json_file in tqdm(json_files, desc=f"Processing JSON in {input_folder}", unit="file"):
        json_path = os.path.join(input_folder, json_file)
        resized_json_path = os.path.join(resized_json_folder, json_file)
        yolo_output_file = os.path.join(yolo_label_folder, os.path.splitext(json_file)[0] + '.txt')

        # 개별 파일 확인: 이미 처리된 JSON과 YOLO 파일이 모두 존재하면 건너뜀
        if os.path.exists(resized_json_path) and os.path.exists(yolo_output_file):
            skipped_json_label += 1
            continue

        try:
            with open(json_path, 'r', encoding='utf-8') as f:
                data = json.load(f)

            # 원본 bbox 좌표
            bbox = data['bboxdata']['bbox_location']
            x1, y1, x2, y2 = map(float, bbox.split(','))
            orig_w, orig_h = original_size
            new_w, new_h = new_size

            # 스케일 및 패딩 계산
            scale = min(new_w / orig_w, new_h / orig_h)
            pad_x = (new_w - orig_w * scale) / 2 # 좌우 패딩
            pad_y = (new_h - orig_h * scale) / 2 # 상하 패딩

            # 리사이즈 및 패딩된 bbox 좌표 계산
            new_x1 = x1 * scale + pad_x
            new_y1 = y1 * scale + pad_y
            new_x2 = x2 * scale + pad_x
            new_y2 = y2 * scale + pad_y

            # 중심 좌표와 크기 계산 및 정규화 (YOLO 라벨 변환을 위함)
            center_x = ((new_x1 + new_x2) / 2) / new_w
            center_y = ((new_y1 + new_y2) / 2) / new_h
            width = (new_x2 - new_x1) / new_w
            height = (new_y2 - new_y1) / new_h

            # YOLO 라벨 저장
            class_id = 0  # 낙상이든 비낙상이든 사람 클래스이므로 항상 0으로 설정 # 팀원 yaml에서는 0을 비낙상, 1을 낙상으로 클래스 설정함
            yolo_result = f"{class_id} {center_x:.6f} {center_y:.6f} {width:.6f} {height:.6f}"
            with open(yolo_output_file, 'w', encoding='utf-8') as f:
                f.write(yolo_result)

            # Resized JSON 저장 (리사이즈된 절대 좌표 반영)
            data['bboxdata']['bbox_location'] = f"{new_x1:.2f},{new_y1:.2f},{new_x2:.2f},{new_y2:.2f}"
            with open(resized_json_path, 'w', encoding='utf-8') as f:
                json.dump(data, f, ensure_ascii=False, indent=4)
            processed_json_label += 1

        except Exception as e:
            print(f"{json_path} 변환에 에러가 발생하였습니다: {e}")

    # 처리 결과 요약 출력
    print(f"'{input_folder}'에서 총 {len(json_files)}개 파일 중 {processed_json_label}개를 처리하고, {skipped_json_label}개를 건너뛰었습니다.")

In [17]:
# JSON 라벨을 YOLO 라벨로 변환, Resizing 반영한 JSON도 생성
print("YOLO 라벨 변환 및 Resizing 반영한 JSON 생성 시작")

datasets = [
    ('Train', train_json, resized_train_json, yolo_train_label),
    ('Val', val_json, resized_val_json, yolo_val_label),
    ('Test', test_json, resized_test_json, yolo_test_label)
]

for dataset_name, input_json_folder, output_resized_json_folder, output_yolo_label_folder in datasets:
    print(f"{dataset_name} 경로의 JSON labels을 처리 중입니다.")
    convert_json_to_yolo_and_resized(
        input_folder=input_json_folder,
        resized_json_folder=output_resized_json_folder,
        yolo_label_folder=output_yolo_label_folder
    )

print("모든 데이터셋 처리가 완료되었습니다.")

YOLO 라벨 변환 및 Resizing 반영한 JSON 생성 시작
Train 경로의 JSON labels을 처리 중입니다.


Processing JSON in D:\Falldown\Dataset\Sorted_Dataset\Train\Label: 100%|██████████| 81650/81650 [00:59<00:00, 1366.17file/s]


Val 경로의 JSON labels을 처리 중입니다.


Processing JSON in D:\Falldown\Dataset\Sorted_Dataset\Val\Label: 100%|██████████| 10200/10200 [00:08<00:00, 1246.08file/s]


Test 경로의 JSON labels을 처리 중입니다.


Processing JSON in D:\Falldown\Dataset\Sorted_Dataset\Test\Label: 100%|██████████| 10230/10230 [00:08<00:00, 1268.82file/s]

모든 데이터셋 처리가 완료되었습니다.
